In [116]:
pip install selectolax

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install navec

Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
import lxml.html
from selectolax.parser import HTMLParser
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from navec import Navec
import warnings
warnings.filterwarnings(action='ignore')

navec_path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(navec_path)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dmitryshilenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dmitryshilenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [105]:
df = pd.read_csv('train_groups.csv')
X = np.reshape(np.array([0] * 37), (1, 37))
for i in np.unique(df['group_id']):
    print(i)
    inds = df[df['group_id'] == i]['doc_id']
    temp_text = []
    for j in inds:
        temp_text.append(get_text('content', j))
    temp_tkns = []
    embeddings = []
    for x in temp_text:
        t = tokenize(x)
        temp_tkns.append(t)
        embeddings.append(doc_2_vec(t))
    mean_embds = []
    for x in embeddings:
        mean_embds.append(np.mean(x, axis=0))
    kmeans_feature = KMeans_dist(mean_embds)
    dbscan_f = dbscan_feature(mean_embds)
    dev_feature = deviation(mean_embds)
    norm_feature = embed_norm(mean_embds)
    top15_feature = top15(temp_tkns)
    top15_binary_feature = top15_feature[0][0]
    top15_float_feature = top15_feature[0][1]
    for i in range(1, len(top15_feature)):
        if len(top15_feature[i][0]) != 15:
            top15_binary_feature = np.vstack((top15_binary_feature, [0]*15))
            top15_float_feature = np.vstack((top15_float_feature, [0]*15))
        else:
            top15_binary_feature = np.vstack((top15_binary_feature, top15_feature[i][0]))
            top15_float_feature = np.vstack((top15_float_feature, top15_feature[i][1]))
    sub_set = np.hstack((np.reshape(kmeans_feature, (kmeans_feature.shape[0], 1)), np.reshape(dbscan_f, (dbscan_f.shape[0], 1)), np.reshape(dev_feature, (dev_feature.shape[0], 1)), np.reshape(norm_feature, (norm_feature.shape[0], 1)), top15_binary_feature, top15_float_feature))
    svm_feature = SVM_fit(sub_set)
    sub_set = np.hstack((sub_set, np.reshape(svm_feature, (svm_feature.shape[0], 1))))
    outlier_feature = feature_local_outlier(sub_set)
    sub_set = np.hstack((sub_set, np.reshape(outlier_feature, (outlier_feature.shape[0], 1))))
    isolation_feature = iso_forest(sub_set)
    sub_set = np.hstack((sub_set, np.reshape(isolation_feature, (isolation_feature.shape[0], 1))))
    X = np.vstack((X, sub_set))
X = X[1:]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129


In [123]:
df = pd.read_csv('test_groups.csv')
X_test = np.reshape(np.array([0] * 37), (1, 37))
for i in np.unique(df['group_id']):
    print(i)
    inds = df[df['group_id'] == i]['doc_id']
    temp_text = []
    for j in inds:
        temp_text.append(get_text('content', j))
    temp_tkns = []
    embeddings = []
    for x in temp_text:
        t = tokenize(x)
        temp_tkns.append(t)
        embeddings.append(doc_2_vec(t))
    mean_embds = []
    for x in embeddings:
        mean_embds.append(np.mean(x, axis=0))
    kmeans_feature = KMeans_dist(mean_embds)
    dbscan_f = dbscan_feature(mean_embds)
    dev_feature = deviation(mean_embds)
    norm_feature = embed_norm(mean_embds)
    top15_feature = top15(temp_tkns)
    top15_binary_feature = top15_feature[0][0]
    top15_float_feature = top15_feature[0][1]
    for i in range(1, len(top15_feature)):
        if len(top15_feature[i][0]) != 15:
            top15_binary_feature = np.vstack((top15_binary_feature, [0]*15))
            top15_float_feature = np.vstack((top15_float_feature, [0]*15))
        else:
            top15_binary_feature = np.vstack((top15_binary_feature, top15_feature[i][0]))
            top15_float_feature = np.vstack((top15_float_feature, top15_feature[i][1]))
    sub_set = np.hstack((np.reshape(kmeans_feature, (kmeans_feature.shape[0], 1)), np.reshape(dbscan_f, (dbscan_f.shape[0], 1)), np.reshape(dev_feature, (dev_feature.shape[0], 1)), np.reshape(norm_feature, (norm_feature.shape[0], 1)), top15_binary_feature, top15_float_feature))
    svm_feature = SVM_fit(sub_set)
    sub_set = np.hstack((sub_set, np.reshape(svm_feature, (svm_feature.shape[0], 1))))
    outlier_feature = feature_local_outlier(sub_set)
    sub_set = np.hstack((sub_set, np.reshape(outlier_feature, (outlier_feature.shape[0], 1))))
    isolation_feature = iso_forest(sub_set)
    sub_set = np.hstack((sub_set, np.reshape(isolation_feature, (isolation_feature.shape[0], 1))))
    X_test = np.vstack((X_test, sub_set))
X_test = X_test[1:]

130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309


In [122]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

y = df_train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
bst = XGBClassifier(n_estimators=401, learning_rate=0.5)
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)
f_score = f1_score(y_test, y_pred)
print(f'F-1 score = {f_score}')

F-1 score = 0.6323987538940811


In [117]:
for x in [0.4 + 0.02 * i for i in range(11)]:
    bst = XGBClassifier(n_estimators=401, learning_rate=x)
    bst.fit(X_train, y_train)
    y_pred = bst.predict(X_test)
    f_score = f1_score(y_test, y_pred)
    print(f'F-1 score with {x} rate = {f_score}')

F-1 score with 0.4 rate = 0.6168674698795181
F-1 score with 0.42000000000000004 rate = 0.6178861788617888
F-1 score with 0.44 rate = 0.6142742582197274
F-1 score with 0.46 rate = 0.6245954692556634
F-1 score with 0.48000000000000004 rate = 0.6198547215496367
F-1 score with 0.5 rate = 0.6368078175895765
F-1 score with 0.52 rate = 0.619281045751634
F-1 score with 0.54 rate = 0.6166394779771615
F-1 score with 0.56 rate = 0.611111111111111
F-1 score with 0.5800000000000001 rate = 0.6060113728675874
F-1 score with 0.6000000000000001 rate = 0.626311541565779


In [124]:
y_test = bst.predict(X_test)

In [132]:
predict = pd.DataFrame(y_test)
predict = predict.reset_index()
predict.columns = ['pair_id', 'target']
predict['pair_id'] += 11691
predict.to_csv('submit.csv', index=False)

In [3]:
def get_text(path: str, doc_id: int):
    f = open(path + '/' + str(doc_id) + '.dat', 'r')
    soup = BeautifulSoup(f, 'html.parser')
    return soup.get_text()

In [4]:
stopwords_ru = nltk.corpus.stopwords.words('russian')
stopwords_en = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("russian")
stopwords_ru.append('это')
tokenizer = RegexpTokenizer(r'\w+')
def tokenize(doc):
    tokens = [i.lower() for i in tokenizer.tokenize(doc) if i.lower() not in stopwords_ru and i.isdigit() == False and len(i) > 1 and i.lower() not in stopwords_en]
    stems = [stemmer.stem(i) for i in tokens]
    return stems

In [5]:
def KMeans_dist(X:np.array):
    centre = np.mean(X, axis=0)
    all_dist = np.sum(np.square(X - centre), axis=1)
    return all_dist

In [6]:
def group_common_words(X):
    arr = np.zeros((len(X), len(X)))
    for i in range(len(X)):
        for j in range(i, len(X)):
            if i == j:
                continue
            else:
                intersect = len(set(tokenize(X[i])).intersection(set(tokenize(X[j]))))
                arr[i, j] = intersect
                arr[j, i] = intersect
    return np.sum(arr, axis=1) / (len(X) - 1)

In [7]:
def doc_2_vec(tokens):
    res = []
    for x in tokens:
        if x in navec:
            res.append(navec[x])
    return res

In [8]:
def dbscan_feature(X):
    clustering = DBSCAN(eps=1).fit(X)
    pred = clustering.labels_
    cast = lambda x: 1 if x == -1 else 0
    vec_cast = np.vectorize(cast)
    pred = vec_cast(pred)
    return pred

In [99]:
def top15(X): # X - токены для каждого документа
    total_top = {}
    every_docs_top = {}
    int_docs_top = []
    j = 0
    for i in X:
        tokens, counts = np.unique(i, return_counts=True)
        counts = counts / len(i)
        sorted_counts = np.argsort(counts)[-15:]
        doc_top_15 = (tokens[sorted_counts], counts[sorted_counts])
        every_docs_top[j] = doc_top_15
        j += 1
        for k in sorted_counts:
            if tokens[k] in total_top:
                total_top[tokens[k]] += counts[k]
            else:
                total_top[tokens[k]] = counts[k]
    total_top = dict(sorted(total_top.items(), key=lambda x: x[1], reverse=True)[:15])
    every_docs_top = list((list(every_docs_top.values())))
    for i in every_docs_top:
        temp_arr = []
        for j in range(len(i[0])):
            if i[0][j] in total_top:
                temp_arr.append(1)
            else:
                temp_arr.append(0)
        int_docs_top.append((temp_arr, i[1]))
    return int_docs_top

In [10]:
def deviation(X):
    mean_vec = np.mean(X, axis=0)
    dev_matr = X - mean_vec
    return np.linalg.norm(dev_matr, axis=1)

In [11]:
def embed_norm(X):
    return np.linalg.norm(X, axis=1)

In [12]:
def SVM_fit(X):
    svm = OneClassSVM(nu = 0.01, kernel = 'rbf', gamma = 'auto').fit(X)
    return svm.score_samples(X)

In [13]:
def feature_local_outlier(X):
    clf = LocalOutlierFactor()
    clf.fit(X)
    return clf.negative_outlier_factor_

In [14]:
def iso_forest(X):
    clf = IsolationForest().fit(X)
    return clf.score_samples(X)

In [ ]:
def SVM_fit(X):
    svm = OneClassSVM(nu = 0.01, kernel = 'rbf', gamma = 'auto').fit(X) 
    return svm.score_samples(X)